In [1]:
from AC_modules.Constructor import *
from Utils import A3C_training
from importlib import reload

In [2]:
# Variable parameters
X = 5
Y = 5
initial = [0,0]
goal = [4,4]
MAX_STEPS = 10

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

In [3]:
relational_HPs = torch.load("Results/Sandbox/Supervised/best_HP_S_chosen-residual_UMUT")
relational_HPs['n_features'] = 16 # reduce because otherwise it crashes
relational_HPs

{'n_kernels': 36,
 'n_features': 16,
 'n_heads': 1,
 'n_attn_modules': 2,
 'feature_hidden_dim': 16,
 'feature_n_residuals': 1}

In [4]:
name = "OheNet"
action_space = 4
#n_features = 16

#HPs = dict(gamma=0.99, tau=0.3, n_steps=5, H=1e-3, **relational_HPs)
HPs = dict(gamma=0.99, tau=0.3, n_steps=5, H=1e-3, n_features=32, map_size=X)
learning_rate = 1e-4

if torch.cuda.is_available():
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'
    
print("Using device "+HPs['device'])
agent_constructor = ActorCriticConstructor(name, True, action_space, **HPs)

Using device cpu
Model:  <class 'AC_modules.Networks.OheNet'>
self.model:  <class 'AC_modules.Networks.OheNet'>
self.shared:  True
self.args:  (4,)
self.kwargs:  {'gamma': 0.99, 'tau': 0.3, 'n_steps': 5, 'H': 0.001, 'n_features': 32, 'map_size': 5, 'device': 'cpu'}


In [7]:
reload(A3C_training)

<module 'Utils.A3C_training' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/Utils/A3C_training.py'>

In [8]:
results = A3C_training.train_sandbox(agent_constructor, learning_rate, game_params, n_training_threads=3, n_episodes=10,
                  max_steps=MAX_STEPS, return_agent=True, random_init=True)

Test process started
Process 1 started
Process 2 started
Process 3 started
All processes started
Test 2 - reward 0.10 - steps to solve 9.20
Episode 10 of process 1 - reward: 0.20 - steps to solve: 8.60
Training process 1 reached maximum episode.
Episode 10 of process 3 - reward: 0.20 - steps to solve: 8.50
Episode 10 of process 2 - reward: 0.30 - steps to solve: 7.60
Training process 2 reached maximum episode.
Training process 3 reached maximum episode.
Test 3 - reward 0.10 - steps to solve 9.30
Putting results in queue
performance (put)  [0.1, 0.1]
steps_to_solve (put) [9.2, 9.3]
performance  [0.1, 0.1]
steps_to_solve  [9.2, 9.3]
critic_losses (put) [tensor(0.0043), tensor(0.0227)]
actor_losses (put)  [tensor(-0.0529), tensor(-0.1265)]
critic_losses  [tensor(0.0043), tensor(0.0227)]
actor_losses  [tensor(-0.0529), tensor(-0.1265)]
entropies (put) [tensor(-0.3056), tensor(-0.2979)]
entropies  [tensor(-0.3056), tensor(-0.2979)]
All processes finished


In [7]:
import torch.multiprocessing as mp

In [27]:
help(mp.Queue().get )

Help on method get in module multiprocessing.queues:

get(block=True, timeout=None) method of multiprocessing.queues.Queue instance



In [9]:
q = mp.Queue()
q.put(1, block=False)

In [10]:
q.get()

1

In [25]:
import time

def wait_and_put(q):
    time.sleep(60)
    q.put([np.arange(1000000)])

In [26]:
q = mp.Queue()
p = mp.Process(target=wait_and_put, args=(q,))
p.start()
result = q.get()
print(result)
p.join()

[array([     0,      1,      2, ..., 999997, 999998, 999999])]
